In [314]:
import numpy as np
import pandas as pd

#random seeds for stochastic parts of neural network 
np.random.seed(10)
from tensorflow import set_random_seed
set_random_seed(15)

from keras.models import Sequential
from keras.layers import Dense, Activation, Merge, Reshape, Dropout
from keras.layers.embeddings import Embedding
from keras.optimizers import SGD
import keras

from sklearn.model_selection import StratifiedKFold
import os

In [315]:
os.chdir('/aichallenge')

In [316]:
import time
t1=time.time()
train = pd.read_table('challengeData.tsv')
test = pd.read_table('scoring_set.tsv')
train_imputed = pd.read_csv('train_imputed.csv')
test_imputed = pd.read_csv('test_imputed.csv')
t2=time.time()
print(t2-t1)

/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,97,98,99,100,104,107,109,112,127,128,129,130,220,221,222) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,54,55,100,104,107,109,112,127,128,129,130,153,171,220,221,222,225,226,227) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


194.551185131073


In [317]:
train.shape

(250000, 237)

In [318]:
train_imputed.isnull().any().any()

False

In [319]:
test_imputed.shape

(1888195, 42)

In [320]:
train_imputed.shape

(226140, 42)

In [321]:
train_imputed = train_imputed.drop('Unnamed: 0',axis=1)
test_imputed = test_imputed.drop('Unnamed: 0',axis=1)

In [322]:
#remove data set name from columns names to make it uniform
t1=time.time()
columns=[]
for col in train.columns:
    columns.append(col.replace('challenge_data.',''))
train.columns=columns
columns=[]
for col in test.columns:
    columns.append(col.replace('scoring_set.',''))
test.columns=columns
t2=time.time()
print(t2-t1)

0.0011782646179199219


In [323]:
train['renewed_yorn'].isnull().sum()

23860

In [324]:
#remove the rows where 'challenge_data.renewed_yorn' is null in training data
t1=time.time()
train=train[train['renewed_yorn'].notnull()]
t2=time.time()
print(t2-t1)

0.812535285949707


In [325]:
#remove the 'challenge_data.renewed_yorn' from training and test data and move it to a seperate variable
t1=time.time()
train_y=train['renewed_yorn']
train = train.drop(['renewed_yorn','zip_code'],axis=1)
test_y=test['renewed_yorn']
test = test.drop(['renewed_yorn','zip_code'],axis=1)
t2=time.time()
print(t2-t1)

6.4706315994262695


In [326]:
#remove columns which have many null values
t1=time.time()
percent = (test.isnull().sum()/test.isnull().count())
test.columns[percent>0.5]
#We would ignore all these 102 columns so we are left with 135 columns
len(test.columns[percent>0.5])
#all_clean_columns=train.columns[percent==0]
#train_clean=train[all_clean_columns]
unclean_columns=test.columns[percent>0.5]
clean_columns=test.columns[percent==0]
train = train.drop(unclean_columns,axis=1)
test = test.drop(unclean_columns,axis=1)
t2=time.time()
print(t2-t1)

67.60910058021545


In [327]:
len(unclean_columns)

36

In [328]:
len(clean_columns)

133

In [329]:
t1=time.time()
percent = (test.isnull().sum()/test.isnull().count())
target_columns=test.columns[percent>0]
train_missing=train[target_columns]
test_missing=test[target_columns]
train = train.drop(target_columns,axis=1)
test = test.drop(target_columns,axis=1)

In [330]:
len(target_columns)

66

In [331]:
#Remove columns which have too many categories and is string type
t1=time.time()
messy_columns = []
for col in target_columns:
    if(train_missing[col].dtype=='O' and len(train_missing[col].unique())>=10):
        messy_columns.append(col)
t2=time.time()
print(t2-t1)

2.252274990081787


In [332]:
# 56 messy columns are removed
len(messy_columns)

25

In [333]:
len(train_missing.columns)

66

In [334]:
t1=time.time()
train_missing=train_missing.drop(messy_columns,axis=1)
test_missing=test_missing.drop(messy_columns,axis=1)
t2=time.time()
print(t2-t1)

1.4593737125396729


In [335]:
#Remove columns which have too many categories and is string type
t1=time.time()
messy_columns = []
for col in train.columns:
    if(train[col].dtype=='O' and len(train[col].unique())>=10):
        messy_columns.append(col)
t2=time.time()
print(t2-t1)

2.2860493659973145


In [336]:
len(messy_columns)

61

In [337]:
t1=time.time()
train=train.drop(messy_columns,axis=1)
test=test.drop(messy_columns,axis=1)
t2=time.time()
print(t2-t1)

2.5496490001678467


In [338]:
train.shape

(226140, 72)

In [339]:
train_missing.shape

(226140, 41)

In [340]:
train_imputed.shape

(226140, 41)

In [341]:
train_imputed.isnull().any().any()

False

In [342]:
train.isnull().any().any()

False

In [343]:
train_copy=train

In [344]:
#Remove the primary key column for data fitting
t1=time.time()
train = train.drop('innovation_challenge_key',axis=1)
test_ids=test['innovation_challenge_key']
test = test.drop('innovation_challenge_key',axis=1)
t2=time.time()
print(t2-t1)

1.3836915493011475


In [32]:
train=train_copy

In [345]:
train.isnull().any().any()

False

In [346]:
train=train.reset_index()

In [347]:
train = train.drop(['index'],axis=1)

In [348]:
for col in train_imputed.columns:
    train[col]=train_imputed[col]
    test[col]=test_imputed[col]

In [349]:
train[col].shape

(226140,)

In [350]:
train.shape

(226140, 112)

In [351]:
test.shape

(1888195, 112)

In [352]:
train.isnull().any().any()

False

In [353]:
#Number of unique values in each columns
t1=time.time()
catcols=[]
scalcols=[]
for col in train.columns:
    if (train[col].dtype !='object'):
        if(len(train[col].unique())<=10):
            catcols.append(col)
        else:
            scalcols.append(col)
t2=time.time()
print(t2-t1)

0.03818345069885254


In [354]:
#handle catcols
t1=time.time()
for col in catcols:
    train[col]=train[col].astype('category')
    test[col]=test[col].astype('category')
t2=time.time()
print(t2-t1)

0.292438268661499


In [355]:
train.isnull().any().sum()

0

In [356]:
test.isnull().any().any()

False

In [357]:
train[catcols].describe()

,product_transaction_type,contract_line_reaction_time_code,sales_hierarchy_level,service_sales_node_base_sales_hierarchy_level
count,226140,226140,226140,226140
unique,3,3,6,6
top,10002,-1,6,6
freq,202522,225296,199960,224170


In [358]:
embed_cols=[]
for col in train.columns:
    if(int(len(train[col].unique()))>2 and int(len(train[col].unique()))<4):
        embed_cols.append(col)

In [359]:
len(embed_cols)

14

In [360]:
train_X = train[embed_cols]
test_X = test[embed_cols]

In [361]:
for col in embed_cols:
    if len(train_X[col].unique())!= len(test_X[col].unique()):
        embed_cols.remove(col)

In [362]:
embed_cols.extend(['contract_line_net_usd_amount','product_net_price'])
train_X = train[embed_cols]
test_X = test[embed_cols]

In [363]:
len(embed_cols)

14

In [364]:
for col in embed_cols:
    print(col)
    print(len(train[col].unique()))
    print(len(test[col].unique()))

product_transaction_type
3
3
contract_line_route_to_market_code
3
3
software_opportunity_type
3
3
service_sales_node_base_country_type
3
3
monetization_type
3
3
product_setup_classification
3
3
discount_category
3
3
service_product_base_business_unit
3
3
global_customer_market_segment_name
3
3
gu_customer_market_segment_name
3
3
country_type
3
3
service_product_base_service_brand_code
3
3
contract_line_net_usd_amount
3699
6950
product_net_price
3985
9581


In [383]:
def build_embedding_network():
    
    models = []
    
    model_cat1 = Sequential()
    model_cat1.add(Embedding(3, 3, input_length=1))
    model_cat1.add(Reshape(target_shape=(3,)))
    models.append(model_cat1)
    
    model_cat2 = Sequential()
    model_cat2.add(Embedding(3, 2, input_length=1))
    model_cat2.add(Reshape(target_shape=(2,)))
    models.append(model_cat2)
    
    model_cat3 = Sequential()
    model_cat3.add(Embedding(3, 2, input_length=1))
    model_cat3.add(Reshape(target_shape=(2,)))
    models.append(model_cat3)
    
    model_cat4 = Sequential()
    model_cat4.add(Embedding(3, 2, input_length=1))
    model_cat4.add(Reshape(target_shape=(2,)))
    models.append(model_cat4)
    
    
    model_cat5 = Sequential()
    model_cat5.add(Embedding(3, 2, input_length=1))
    model_cat5.add(Reshape(target_shape=(2,)))
    models.append(model_cat5)
    
    
    model_cat6 = Sequential()
    model_cat6.add(Embedding(3, 2, input_length=1))
    model_cat6.add(Reshape(target_shape=(2,)))
    models.append(model_cat6)
    
    
    model_cat7 = Sequential()
    model_cat7.add(Embedding(3, 2, input_length=1))
    model_cat7.add(Reshape(target_shape=(2,)))
    models.append(model_cat7)
    
    
    model_cat8 = Sequential()
    model_cat8.add(Embedding(3, 2, input_length=1))
    model_cat8.add(Reshape(target_shape=(2,)))
    models.append(model_cat8)
    
    
    model_cat9 = Sequential()
    model_cat9.add(Embedding(3, 2, input_length=1))
    model_cat9.add(Reshape(target_shape=(2,)))
    models.append(model_cat9)
    
    
    model_cat10 = Sequential()
    model_cat10.add(Embedding(3, 2, input_length=1))
    model_cat10.add(Reshape(target_shape=(2,)))
    models.append(model_cat10)
    
    
    model_cat11 = Sequential()
    model_cat11.add(Embedding(3, 2, input_length=1))
    model_cat11.add(Reshape(target_shape=(2,)))
    models.append(model_cat11)
    
    
    model_cat12 = Sequential()
    model_cat12.add(Embedding(3, 2, input_length=1))
    model_cat12.add(Reshape(target_shape=(2,)))
    models.append(model_cat12)
    
    
    #model_rest = Sequential()
    #model_rest.add(Dense(16, input_dim=2))
    #models.append(model_rest)
    
    model = Sequential()
    model.add(Merge(models, mode='concat'))
    model.add(Dense(80))
    model.add(Activation('relu'))
    model.add(Dropout(.35))
    model.add(Dense(20))
    model.add(Activation('relu'))
    model.add(Dropout(.15))
    model.add(Dense(10))
    model.add(Activation('relu'))
    model.add(Dropout(.15))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(optimizer=SGD(lr=0.002, momentum=0.0, decay=0.0, nesterov=True), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model
    

In [380]:
train_X.shape

(226140, 14)

In [367]:
test_X.shape

(1888195, 14)

In [368]:
y_train = pd.get_dummies(train_y)

In [369]:
embed_cols.remove('contract_line_net_usd_amount')
embed_cols.remove('product_net_price')

In [370]:
def preproc(X_train, X_val, X_test):

    input_list_train = []
    input_list_val = []
    input_list_test = []
    
    #the cols to be embedded: rescaling to range [0, # values)
    for c in embed_cols:
        raw_vals = np.unique(X_train[c])
        val_map = {}
        for i in range(len(raw_vals)):
            val_map[raw_vals[i]] = i       
        input_list_train.append(X_train[c].map(val_map).values)
        input_list_val.append(X_val[c].map(val_map).fillna(0).values)
        input_list_test.append(X_test[c].map(val_map).fillna(0).values)
     
    #the rest of the columns
    #other_cols = [c for c in X_train.columns if (not c in embed_cols)]
    #input_list_train.append(X_train[other_cols].values)
    #input_list_val.append(X_val[other_cols].values)
    #input_list_test.append(X_test[other_cols].values)
    
    return input_list_train, input_list_val, input_list_test    


In [371]:
def ginic(actual, pred):
    n = len(actual)
    a_s = actual[np.argsort(pred)]
    a_c = a_s.cumsum()
    giniSum = a_c.sum() / a_c[-1] - (n + 1) / 2.0
    return giniSum / n

In [372]:
def gini_normalizedc(a, p):
    return ginic(a, p) / ginic(a, a)

In [373]:
K=8
runs_per_fold = 3
n_epochs = 15

cv_ginis = []
full_val_preds = np.zeros(np.shape(train_X)[0])
y_preds = np.zeros((np.shape(test_X)[0],K))

kfold = StratifiedKFold(n_splits = K, 
                            random_state = 231, 
                            shuffle = True)  

In [374]:
y_train=y_train['Y']

In [384]:
for i, (f_ind, outf_ind) in enumerate(kfold.split(train_X, y_train)):

    X_train_f, X_val_f = train_X.loc[f_ind].copy(), train_X.loc[outf_ind].copy()
    y_train_f, y_val_f = y_train[f_ind], y_train[outf_ind]
    
    X_test_f = test_X.copy()
    
    #upsampling adapted from kernel: 
    #https://www.kaggle.com/ogrellier/xgb-classifier-upsampling-lb-0-283
    pos = (pd.Series(y_train_f == 1))
    
    # Add positive examples
    X_train_f = pd.concat([X_train_f, X_train_f.loc[pos]], axis=0)
    y_train_f = pd.concat([y_train_f, y_train_f.loc[pos]], axis=0)
    
    # Shuffle data
    idx = np.arange(len(X_train_f))
    np.random.shuffle(idx)
    X_train_f = X_train_f.iloc[idx]
    y_train_f = y_train_f.iloc[idx]
    
    #preprocessing
    proc_X_train_f, proc_X_val_f, proc_X_test_f = preproc(X_train_f, X_val_f, X_test_f)
    
    #track oof prediction for cv scores
    val_preds = 0
    runs_per_fold=1
    for j in range(runs_per_fold):
    
        NN = build_embedding_network()
        #print(len(proc_X_train_f))
        NN.fit(proc_X_train_f, y_train_f.values, epochs=1,  verbose=3)
               #callbacks=[keras.callbacks.TensorBoard(log_dir="logs/final/{}".format(time()), histogram_freq=1, write_graph=True, write_images=True)])
   
        val_preds += NN.predict(proc_X_val_f)[:,0] / runs_per_fold
        y_preds[:,i] += NN.predict(proc_X_test_f)[:,0] / runs_per_fold
        
    full_val_preds[outf_ind] += val_preds
        
    cv_gini = gini_normalizedc(y_val_f.values, val_preds)
    cv_ginis.append(cv_gini)
    print ('\nFold %i prediction cv gini: %.5f\n' %(i,cv_gini))

/usr/lib64/python3.6/site-packages/pandas/core/series.py:696: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:79: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Epoch 1/1

Fold 0 prediction cv gini: nan

Epoch 1/1

Fold 1 prediction cv gini: nan

Epoch 1/1

Fold 2 prediction cv gini: nan

Epoch 1/1

Fold 3 prediction cv gini: nan

Epoch 1/1

Fold 4 prediction cv gini: nan

Epoch 1/1

Fold 5 prediction cv gini: nan

Epoch 1/1

Fold 6 prediction cv gini: nan

Epoch 1/1

Fold 7 prediction cv gini: nan



In [311]:
print('Mean out of fold gini: %.5f' % np.mean(cv_ginis))
print('Full validation gini: %.5f' % gini_normalizedc(y_train.values, full_val_preds))
X_train_pp = preproc(test_X,train_X,train_X)
pred = NN.predict(X_train_pp[0])[:,0]
#y_pred_final = np.mean(y_preds, axis=1)

submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test_ids
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('NN_EntityEmbed_10fold-sub.csv',index=False)

Mean out of fold gini: nan
Full validation gini: -0.03344


In [312]:
len(pred)

1888195

In [313]:
pred

array([nan, nan, nan, ..., nan, nan, nan], dtype=float32)

In [116]:
from sklearn.metrics import log_loss

In [ ]:
y_pred_final